In [36]:
import torch
import torchvision

import numpy as np

from torch.utils.data import Dataset


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cpu for inference


In [37]:
num_epochs = 4
batch_size = 4
learning_rate = 0.001

# Czemu .5 ?
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((.5, .5, .5), (.5, .5, .5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [38]:
class ConvNet(torch.nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 6, 5)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.fc1 = torch.nn.Linear(16 * 5 * 5, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(torch.nn.functional.relu(self.conv1(x)))
        x = self.pool(torch.nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [39]:
model = ConvNet().to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [40]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')

Epoch [1/4], Step [2000/12500], Loss: 2.2897
Epoch [1/4], Step [4000/12500], Loss: 2.3330
Epoch [1/4], Step [6000/12500], Loss: 2.2795
Epoch [1/4], Step [8000/12500], Loss: 2.2906
Epoch [1/4], Step [10000/12500], Loss: 2.1908
Epoch [1/4], Step [12000/12500], Loss: 2.2097
Epoch [2/4], Step [2000/12500], Loss: 2.1175
Epoch [2/4], Step [4000/12500], Loss: 2.2810
Epoch [2/4], Step [6000/12500], Loss: 1.6792
Epoch [2/4], Step [8000/12500], Loss: 1.7351
Epoch [2/4], Step [10000/12500], Loss: 2.1172
Epoch [2/4], Step [12000/12500], Loss: 1.5363
Epoch [3/4], Step [2000/12500], Loss: 2.2107
Epoch [3/4], Step [4000/12500], Loss: 2.0335
Epoch [3/4], Step [6000/12500], Loss: 1.5082
Epoch [3/4], Step [8000/12500], Loss: 1.6299
Epoch [3/4], Step [10000/12500], Loss: 2.5952
Epoch [3/4], Step [12000/12500], Loss: 1.1283
Epoch [4/4], Step [2000/12500], Loss: 2.2579
Epoch [4/4], Step [4000/12500], Loss: 1.9384
Epoch [4/4], Step [6000/12500], Loss: 1.4526
Epoch [4/4], Step [8000/12500], Loss: 1.5389
Epoc